# EIA 860M Update Inspection

To run this notebook, you need to refresh the changelog data first, by updating parameters:
- In `.env`, change `PUDL_VERSION` to the latest found [here](https://github.com/catalyst-cooperative/pudl/releases)
- In `src/constants.py`, set `PUDL_LATEST_YEAR` to the latest year for which PUDL has complete data.

and running `make all`.

In [1]:
import pandas as pd

from dbcp.helpers import get_sql_engine

engine = get_sql_engine()

with engine.connect() as con:
    eia860m = pd.read_sql_table("pudl_eia860m_changelog", con, schema="data_warehouse")

In [4]:
END_OF_PREVIOUS_QUARTER_DATE = "2024-10-01"

def pct_change_mw_by_status(df):
    recent_quarter = df.loc[df.groupby(["generator_id", "plant_id_eia"])['report_date'].idxmax()]
    
    recent_quarter_mw_by_status = recent_quarter.groupby("operational_status_code").capacity_mw.sum()

    previous_quarter = df[df.report_date < END_OF_PREVIOUS_QUARTER_DATE]
    previous_quarter = previous_quarter.loc[previous_quarter.groupby(["generator_id", "plant_id_eia"])['report_date'].idxmax()]
    previous_quarter_mw_by_status = previous_quarter.groupby("operational_status_code").capacity_mw.sum()

    return ((recent_quarter_mw_by_status - previous_quarter_mw_by_status) / previous_quarter_mw_by_status) * 100
    

In [5]:
pct_change_mw_by_status(eia860m)

operational_status_code
1     10.430908
2      6.532631
3      4.905398
4     -5.174416
5      9.569163
6     12.365107
7      0.714077
8      0.079723
99     0.000000
Name: capacity_mw, dtype: float64

## Capacity by status by ISO

In [6]:
ISO_REGIONS = ("MISO", "PJM", "CISO", "ERCO", "ISNE", "NYIS", "SWPP") 

In [7]:
eia860_isos = eia860m[eia860m.balancing_authority_code_eia.isin(ISO_REGIONS)]


for region in ISO_REGIONS:
    print(region)
    pct_change = pct_change_mw_by_status(eia860_isos[eia860_isos["balancing_authority_code_eia"] == region])
    print(pct_change)
    print()

MISO
operational_status_code
1     0.848435
2    22.710135
3    13.838777
4    10.811155
5    18.757170
6    15.527606
7     0.771333
8     0.050665
Name: capacity_mw, dtype: float64

PJM
operational_status_code
1       9.898398
2       1.209212
3      -0.003964
4       1.255510
5      -0.968126
6     209.016394
7       0.163477
8       0.101088
99      0.000000
Name: capacity_mw, dtype: float64

CISO
operational_status_code
1      7.932810
2     -0.425961
3     25.064151
4     -8.198664
5      3.443937
6      0.943523
7      1.891107
8     -0.076096
99     0.000000
Name: capacity_mw, dtype: float64

ERCO
operational_status_code
1      9.219364
2      9.677581
3      8.698993
4     -1.511497
5    -19.986722
6      6.414715
7      2.277254
8      0.000000
99     0.000000
Name: capacity_mw, dtype: float64

ISNE
operational_status_code
1       0.000000
2     -37.193092
3      -1.548947
4     189.530504
5       1.449929
6     -82.614942
7       1.141464
8      -1.600243
99      0.000000
Na